In [8]:
import pandas as pd
import cx_Oracle

In [9]:
#Connect to Quorum DB
dsnStr = cx_Oracle.makedsn("lonodsou01-sc.uk.db.com", "1721", service_name = "LNDBP1U.uk.db.com")
ora_con = cx_Oracle.connect(user="DBP_110_RAP_USER", password="London2012!", dsn=dsnStr)
print(ora_con.version)#Remove once complete

11.2.0.4.0


In [10]:
#ODS Book Hierarchy 
querybook = """select * from V_ORG_HRRCHY 
                where TBK_NM in ('GPFFRMOTEST-HKS0','GPFFRMOTEST-XCB0','DBAG-HKN-80000000-STG','DBAG-HKN-80500000-STG','DBAG-HKN-81000000-STG','DBAG-HKN-81500000-STG','DBAG-HKN-88880000-STG')"""
df_book = pd.read_sql(querybook, con=ora_con) 

In [11]:
df_book

,BUSINESS_LINE,SUB_BUSINESS,UBR1_ALIAS,UBR1_NM,UBR2_ALIAS,UBR2_NM,UBR3_ALIAS,UBR3_NM,UBR4_ALIAS,UBR4_NM,...,DSK1_ALIAS,DSK1_NM,DSK5_ALIAS,DSK5_NM,PRTFL_ALIAS,PRTFL_NM,TBK_ALIAS,TBK_DMN_NM,TBK_NM,TBK_ORG_SID
0,Global Prime Finance,Prime Finance - APAC,95,Group (aggregated),183,Total Divisions and Infrastructure,4596,Operating Business,2482,Corporate Banking & Securities,...,Flow Swaps - Asia,Flow Swaps - Asia,Flow Swaps - Asia,Flow Swaps - Asia,Hybrid Swaps - Asia,Hybrid Swaps - Asia,GPFFRMOTEST-HKS0,FR-BOOK,GPFFRMOTEST-HKS0,2074440693
1,Global Prime Finance,Prime Finance - APAC,95,Group (aggregated),183,Total Divisions and Infrastructure,4596,Operating Business,2482,Corporate Banking & Securities,...,Flow Swaps - Asia,Flow Swaps - Asia,Flow Swaps - Asia,Flow Swaps - Asia,Hybrid Swaps - Asia,Hybrid Swaps - Asia,GPFFRMOTEST-XCB0,FR-BOOK,GPFFRMOTEST-XCB0,2074440696
2,Global Prime Finance,Prime Finance - APAC,95,Group (aggregated),183,Total Divisions and Infrastructure,4596,Operating Business,2482,Corporate Banking & Securities,...,Flow Swaps - Asia,Flow Swaps - Asia,Flow Swaps - Asia,Flow Swaps - Asia,Hybrid Swaps - Asia,Hybrid Swaps - Asia,DBAG-HKN-80000000-STG,FR-BOOK,DBAG-HKN-80000000-STG,2074710623
3,Global Prime Finance,Prime Finance - APAC,95,Group (aggregated),183,Total Divisions and Infrastructure,4596,Operating Business,2482,Corporate Banking & Securities,...,Flow Swaps - Asia,Flow Swaps - Asia,Flow Swaps - Asia,Flow Swaps - Asia,Hybrid Swaps - Asia,Hybrid Swaps - Asia,DBAG-HKN-80500000-STG,FR-BOOK,DBAG-HKN-80500000-STG,2074710624
4,Global Prime Finance,Prime Finance - APAC,95,Group (aggregated),183,Total Divisions and Infrastructure,4596,Operating Business,2482,Corporate Banking & Securities,...,Flow Swaps - Asia,Flow Swaps - Asia,Flow Swaps - Asia,Flow Swaps - Asia,Hybrid Swaps - Asia,Hybrid Swaps - Asia,DBAG-HKN-81000000-STG,FR-BOOK,DBAG-HKN-81000000-STG,2074710625
5,Global Prime Finance,Prime Finance - APAC,95,Group (aggregated),183,Total Divisions and Infrastructure,4596,Operating Business,2482,Corporate Banking & Securities,...,Flow Swaps - Asia,Flow Swaps - Asia,Flow Swaps - Asia,Flow Swaps - Asia,Hybrid Swaps - Asia,Hybrid Swaps - Asia,DBAG-HKN-81500000-STG,FR-BOOK,DBAG-HKN-81500000-STG,2074710626
6,Global Prime Finance,Prime Finance - APAC,95,Group (aggregated),183,Total Divisions and Infrastructure,4596,Operating Business,2482,Corporate Banking & Securities,...,Flow Swaps - Asia,Flow Swaps - Asia,Flow Swaps - Asia,Flow Swaps - Asia,Hybrid Swaps - Asia,Hybrid Swaps - Asia,DBAG-HKN-88880000-STG,FR-BOOK,DBAG-HKN-88880000-STG,2074710627


In [12]:
#ODS Raw Data
queryods = """WITH ds AS (SELECT
                     dset.ORG_SID,
                     org.ORG_ALIAS,
                     org.ORG_DMN_NM,
                     org.ORG_TYP_NM,
                     dset.DSET_COB_DT AS COB_DT,
                     dset.DSET_SID,
                     dstyp.DSTYP_NM,
                     vltyp.VLTYP_NM,
                     dsdef.DSDEF_SUB_TYP_NM,
                     dstyp.DSTYP_IS_EXTERNAL_IN,
                     dstyp.DSTYP_IS_ADJUSTMENT_IN,
                     dstyp.DSTYP_IS_AGGREGATE_IN,
                     dset.DSET_VERSN_ID,
                     dset.DSET_SRC_CREATION_TS,
                     dset.VLMD_SID,
                     dset.TRCLP_REC_CNT_NO,
                     DENSE_RANK() OVER (
                       PARTITION BY dset.ORG_SID, dset.DSET_COB_DT, dstyp.DSTYP_NM, vltyp.VLTYP_NM, dsdef.DSDEF_SUB_TYP_NM
                       ORDER BY dset.DSET_TTSTART_TS DESC) AS RNK_DSET
                FROM T_DSET_DATASET dset
                     JOIN M_DSDEF_DATASET_DEF dsdef                  ON dset.DSDEF_SID = dsdef.DSDEF_SID
                     JOIN M_ORG_ORGANISATION org                  ON dset.ORG_SID = org.ORG_SID
                     JOIN M_DSTYP_DATA_SET_TYP dstyp                 ON dsdef.DSTYP_SID = dstyp.DSTYP_SID
                     LEFT JOIN M_VLTYP_VLTN_TYP vltyp                ON dsdef.VLTYP_SID = vltyp.VLTYP_SID
                     
               WHERE 1=1
        AND ORG_ALIAS 
        IN
        
        (select TBK_ALIAS from V_ORG_HRRCHY 

where ORG_ALIAS in ('GPFFRMOTEST-HKS0','GPFFRMOTEST-XCB0','DBAG-HKN-80000000-STG','DBAG-HKN-80500000-STG','DBAG-HKN-81000000-STG','DBAG-HKN-81500000-STG','DBAG-HKN-88880000-STG'))--('6373','6654','6750')){{^UBR_ALIAS^}}
        AND DSET_COB_DT = TO_DATE('2019-02-07','YYYY-MM-DD') 
        AND DSTYP_IS_EXTERNAL_IN = 'Y'
        AND VLTYP_NM = 'EOD Official')  ,        
           
         SEC AS (  SELECT inst_sec.INST_SID,
                      inst_sec.INST_SID AS SEC_SID,
                      inst_sec.INST_ALIAS AS SEC_ALIAS,
                      inst_sec.INST_DMN_NM AS SEC_DMN_NM,
                      inst_sec.INST_NM AS SEC_NM,
                      SEC_DB_SEC_TYP_NM
                 FROM M_SEC_SECURITY sec
                      INNER JOIN M_INST_INSTRUMENT inst_sec
                         ON (sec.SEC_SID = inst_sec.INST_SID
                             AND inst_sec.INST_TTSTATUS = 'C')
                      LEFT OUTER JOIN M_CCY_CURRENCY hldg
                         ON sec.HLDG_CCY_SID = hldg.CCY_SID
                      LEFT OUTER JOIN M_INST_INSTRUMENT inst_hldg
                         ON hldg.CCY_SID = inst_hldg.INST_SID
                      LEFT OUTER JOIN M_CCY_CURRENCY undrlyng
                         ON sec.UNDRLYNG_CCY_SID = undrlyng.CCY_SID
                      LEFT OUTER JOIN M_INST_INSTRUMENT inst_underlyng
                         ON undrlyng.CCY_SID = inst_underlyng.INST_SID
             ORDER BY inst_sec.INST_ALIAS )
  SELECT ds.ORG_ALIAS,
         ds.ORG_DMN_NM,
         ds.ORG_TYP_NM,
         ds.COB_DT,
         ds.DSET_SID,
         ds.DSTYP_NM,
         ds.VLTYP_NM,
         ds.DSDEF_SUB_TYP_NM,
         ds.DSTYP_IS_EXTERNAL_IN,
         ds.DSTYP_IS_ADJUSTMENT_IN,
         ds.DSTYP_IS_AGGREGATE_IN,
         btrd.BTRD_ALIAS,
         btrd.BTRD_DMN_NM,
         btrd.BTRD_TYP_NM,
         btrd.BTRD_VERSN_ID,
         pdef_pdtyp.PDTYP_ALIAS AS PDEF_PRODUCT_TYPE,
         NVL (pdtyp1.PDTYP_ALIAS, pdtyp2.PDTYP_ALIAS) AS RISK_PDTYP_ALIAS,
         loc1.LOC_ALIAS AS SALES_LOC_ALIAS,
         loc2.LOC_ALIAS AS BRNCH_LOC_ALIAS,
         trdsi.RVNS_SID,
         inst2.INST_ALIAS,
         pdef.ADJ_SID,
         'BALANCE_SHEET' AS AMTYP_NM,
         patyp.PATYP_ALIAS,
         bsamt.BSAMT_TXC_AM,
         inst1.INST_ALIAS AS TXC_CCY,
         bsamt.BSAMT_LFC_AM,
         inst3.INST_ALIAS AS LFC_CCY,
         bsamt.BSAMT_GRC_AM,
         inst4.INST_ALIAS AS GRC_CCY,
        sec.SEC_DB_SEC_TYP_NM as DB_SEC_TYP
    FROM DS
         JOIN T_BSAMT_BS_AMT bsamt                   ON ds.DSET_SID = bsamt.DSET_SID
         JOIN T_PDEF_PSTN_DEF pdef                   ON bsamt.PDEF_SID = pdef.PDEF_SID
         LEFT JOIN T_BTRD_BASE_TRADE btrd            ON pdef.BTRD_SID = btrd.BTRD_SID
         LEFT JOIN T_CTRD_CASH_EVENT ctrd            ON btrd.BTRD_SID = ctrd.CTRD_SID
         LEFT JOIN T_RTRD_RISK_TRADE rtrd            ON NVL (ctrd.BTRD_SID, btrd.BTRD_SID) = rtrd.RTRD_SID
         LEFT JOIN M_PDTYP_PRDCT_TYP pdtyp1          ON rtrd.RTRD_RISK_PDTYP_SID = pdtyp1.PDTYP_SID
         LEFT JOIN M_PDTYP_PRDCT_TYP pdef_pdtyp            ON pdef.PDTYP_SID = pdef_pdtyp.PDTYP_SID
         LEFT JOIN T_PREM_PREMIUM prem               ON NVL (ctrd.BTRD_SID, btrd.BTRD_SID) = prem.PREM_SID
         LEFT JOIN M_PDTYP_PRDCT_TYP pdtyp2          ON prem.PREM_RISK_PDTYP_SID = pdtyp2.PDTYP_SID
         LEFT JOIN T_TRD_TRADE trd                   ON NVL (rtrd.TRD_SID, prem.TRD_SID) = trd.TRD_SID
         LEFT JOIN T_TRDSI_TRD_SALES_INFO trdsi      ON (trd.TRD_SID = trdsi.TRD_SID AND trdsi.TRDSI_TTSTATUS = 'C')
         LEFT JOIN M_LOC_LOCATION loc1               ON trdsi.SALES_LOC_SID = loc1.LOC_SID
         LEFT JOIN M_LOC_LOCATION loc2               ON trdsi.BRNCH_LOC_SID = loc2.LOC_SID
         LEFT JOIN M_INST_INSTRUMENT inst2           ON pdef.INST_SID = inst2.INST_SID
         JOIN M_BSTYP_BS_AMT_TYP bstyp               ON bsamt.BSTYP_SID = bstyp.BSTYP_SID
         JOIN M_PATYP_PSTN_AMT_TYP patyp             ON bstyp.PATYP_SID = patyp.PATYP_SID
         LEFT JOIN M_CCY_CURRENCY ccy1               ON bsamt.TXC_CCY_SID = ccy1.CCY_SID
         LEFT JOIN M_INST_INSTRUMENT inst1           ON ccy1.CCY_SID = inst1.INST_SID
         LEFT JOIN M_CCY_CURRENCY ccy3               ON bsamt.LFC_CCY_SID = ccy3.CCY_SID
         LEFT JOIN M_INST_INSTRUMENT inst3           ON ccy3.CCY_SID = inst3.INST_SID
         LEFT JOIN M_CCY_CURRENCY ccy4               ON bsamt.GRC_CCY_SID = ccy4.CCY_SID
         LEFT JOIN M_INST_INSTRUMENT inst4           ON ccy4.CCY_SID = inst4.INST_SID         
         LEFT OUTER JOIN M_SEC_SECURITY sec           ON trd.INST_SID = sec.SEC_SID
         LEFT OUTER JOIN M_INST_INSTRUMENT inst_sec           ON sec.SEC_SID = inst_sec.INST_SID
         LEFT OUTER JOIN SEC           ON NVL (pdef.INST_SID, trd.INST_SID) = sec.INST_SID 
     WHERE RNK_DSET = 1
ORDER BY ORG_ALIAS, COB_DT, DSTYP_IS_EXTERNAL_IN, DSTYP_IS_AGGREGATE_IN, BTRD_ALIAS, BTRD_VERSN_ID, DSET_SID
"""
df_ods = pd.read_sql(queryods, con=ora_con) 

In [13]:
df_ods

,ORG_ALIAS,ORG_DMN_NM,ORG_TYP_NM,COB_DT,DSET_SID,DSTYP_NM,VLTYP_NM,DSDEF_SUB_TYP_NM,DSTYP_IS_EXTERNAL_IN,DSTYP_IS_ADJUSTMENT_IN,...,ADJ_SID,AMTYP_NM,PATYP_ALIAS,BSAMT_TXC_AM,TXC_CCY,BSAMT_LFC_AM,LFC_CCY,BSAMT_GRC_AM,GRC_CCY,DB_SEC_TYP
0,DBAG-HKN-80000000-STG,FR-BOOK,TBK,2019-02-07,2454199987,Valuation,EOD Official,Flash,Y,N,...,None,BALANCE_SHEET,PV MTM,0.000000e+00,HKD,None,None,None,None,S
1,DBAG-HKN-80000000-STG,FR-BOOK,TBK,2019-02-07,2454199987,Valuation,EOD Official,Flash,Y,N,...,None,BALANCE_SHEET,PV Cost,0.000000e+00,HKD,None,None,None,None,S
2,DBAG-HKN-80000000-STG,FR-BOOK,TBK,2019-02-07,2454199987,Valuation,EOD Official,Flash,Y,N,...,None,BALANCE_SHEET,Model Cash Settled Margin,0.000000e+00,HKD,None,None,None,None,S
3,DBAG-HKN-80000000-STG,FR-BOOK,TBK,2019-02-07,2454199987,Valuation,EOD Official,Flash,Y,N,...,None,BALANCE_SHEET,PV MTM Margin,0.000000e+00,HKD,None,None,None,None,S
4,DBAG-HKN-80500000-STG,FR-BOOK,TBK,2019-02-07,2454199988,Valuation,EOD Official,Flash,Y,N,...,None,BALANCE_SHEET,PV MTM,0.000000e+00,HKD,None,None,None,None,S
5,DBAG-HKN-80500000-STG,FR-BOOK,TBK,2019-02-07,2454199988,Valuation,EOD Official,Flash,Y,N,...,None,BALANCE_SHEET,PV Cost,0.000000e+00,HKD,None,None,None,None,S
6,DBAG-HKN-80500000-STG,FR-BOOK,TBK,2019-02-07,2454199988,Valuation,EOD Official,Flash,Y,N,...,None,BALANCE_SHEET,Model Cash Settled Margin,0.000000e+00,HKD,None,None,None,None,S
7,DBAG-HKN-80500000-STG,FR-BOOK,TBK,2019-02-07,2454199988,Valuation,EOD Official,Flash,Y,N,...,None,BALANCE_SHEET,PV MTM Margin,0.000000e+00,HKD,None,None,None,None,S
8,DBAG-HKN-81000000-STG,FR-BOOK,TBK,2019-02-07,2454199989,Valuation,EOD Official,Flash,Y,N,...,None,BALANCE_SHEET,PV MTM,0.000000e+00,HKD,None,None,None,None,S
9,DBAG-HKN-81000000-STG,FR-BOOK,TBK,2019-02-07,2454199989,Valuation,EOD Official,Flash,Y,N,...,None,BALANCE_SHEET,PV Cost,0.000000e+00,HKD,None,None,None,None,S


In [14]:
df_ods.to_csv('df_ods.csv', index=False)